In [158]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional,GRU,SpatialDropout1D,SimpleRNN
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks

In [143]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [144]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))

Loading train and test data
... 
Data loaded


In [145]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [146]:

# Building the model
print(colored("Creating the LSTM model", "yellow"))
modellstm = Sequential()
modellstm.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modellstm.add(SpatialDropout1D(0.4))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modellstm.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = False))

modellstm.add(Dense(3, activation = 'softmax'))
modellstm.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modellstm.summary()

Creating the LSTM model
Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 37, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 37, 256)           394240    
_________________________________________________________________
lstm_22 (LSTM)               (None, 37, 256)           525312    
_________________________________________________________________
lstm_23 (LSTM)               (None, 37, 256)           525312    
_________________________________________________________________
lstm_24 (LSTM)               (None, 37, 256)           525312    
_________________________________________________________________
lstm_25 (LSTM)               

In [147]:
# Training the model
print(colored("Training the LSTM model", "green"))
history = modellstm.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the LSTM model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 340s 16ms/step - loss: 0.9195 - accuracy: 0.5401
Epoch 2/3
20741/20741 [==============================] - 342s 17ms/step - loss: 0.7819 - accuracy: 0.6262
Epoch 3/3
20741/20741 [==============================] - 340s 16ms/step - loss: 0.6666 - accuracy: 0.7109


In [148]:
print(colored("Testing the LSTM model", "green"))

loss, accuracy = modellstm.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x=accuracy
score, accuracy = modellstm.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y=accuracy


Testing the LSTM model
20741/20741 [==============================] - 95s 5ms/step
Train accuracy: 0.7634636759757996
1749/1749 [==============================] - 8s 5ms/step
Test accuracy: 0.7364208102226257


In [149]:
# Building the model
print(colored("Creating the GRU model", "yellow"))
modelgru = Sequential()
modelgru.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelgru.add(SpatialDropout1D(0.4))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
modelgru.add(GRU(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = False))

modelgru.add(Dense(3, activation = 'softmax'))
modelgru.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelgru.summary()

Creating the GRU model
Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 37, 128)           0         
_________________________________________________________________
gru_3 (GRU)                  (None, 37, 256)           295680    
_________________________________________________________________
gru_4 (GRU)                  (None, 37, 256)           393984    
_________________________________________________________________
gru_5 (GRU)                  (None, 37, 256)           393984    
_________________________________________________________________
gru_6 (GRU)                  (None, 37, 256)           393984    
_________________________________________________________________
gru_7 (GRU)                  (

In [150]:
# Training the model
print(colored("Training the GRU model", "green"))
history = modelgru.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the GRU model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 351s 17ms/step - loss: 1407594.0326 - accuracy: 0.4251
Epoch 2/3
20741/20741 [==============================] - 349s 17ms/step - loss: 103313.9306 - accuracy: 0.4197
Epoch 3/3
20741/20741 [==============================] - 350s 17ms/step - loss: 409289.6453 - accuracy: 0.4232


In [151]:
print(colored("Testing the GRU model", "green"))

loss, accuracy = modelgru.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x1=accuracy
score, accuracy = modelgru.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y1=accuracy

Testing the GRU model
20741/20741 [==============================] - 79s 4ms/step
Train accuracy: 0.5352200865745544
1749/1749 [==============================] - 7s 4ms/step
Test accuracy: 0.5397369861602783


In [152]:
# 1. define the network
print(colored("Creating the Bidirectional LSTM model", "yellow"))

modelblstm = Sequential()
modelblstm.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))

modelblstm.add(Bidirectional(LSTM(128)))

modelblstm.add(Dropout(0.1))

modelblstm.add(Dense(3, activation = 'softmax'))
modelblstm.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelblstm.summary()

Creating the Bidirectional LSTM model
Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 256)               263168    
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 771       
Total params: 519,939
Trainable params: 519,939
Non-trainable params: 0
_________________________________________________________________


In [153]:

# Training the model
print(colored("Training the Bidirectional LSTM model", "green"))
history = modelblstm.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Bidirectional LSTM model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 52s 3ms/step - loss: 0.7936 - accuracy: 0.6223
Epoch 2/3
20741/20741 [==============================] - 51s 2ms/step - loss: 0.5761 - accuracy: 0.7505
Epoch 3/3
20741/20741 [==============================] - 51s 2ms/step - loss: 0.5226 - accuracy: 0.7768


In [154]:

print(colored("Testing the Bidirectional LSTM model", "green"))

loss, accuracy = modelblstm.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x2=accuracy
score, accuracy = modelblstm.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y2=accuracy

Testing the Bidirectional LSTM model
20741/20741 [==============================] - 13s 650us/step
Train accuracy: 0.8055059909820557
1749/1749 [==============================] - 1s 666us/step
Test accuracy: 0.7764436602592468


In [155]:
# 1. define the network
print(colored("Creating the Bidirectional GRU model", "yellow"))

modelbgru = Sequential()
modelbgru.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))

modelbgru.add(Bidirectional(GRU(128)))

modelbgru.add(Dropout(0.1))

modelbgru.add(Dense(3, activation = 'softmax'))
modelbgru.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelbgru.summary()

Creating the Bidirectional GRU model
Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 256)               197376    
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 771       
Total params: 454,147
Trainable params: 454,147
Non-trainable params: 0
_________________________________________________________________


In [156]:
# Training the model
print(colored("Training the Bidirectional GRU model", "green"))
history = modelbgru.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Bidirectional GRU model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 53s 3ms/step - loss: 0.7787 - accuracy: 0.6263
Epoch 2/3
20741/20741 [==============================] - 52s 2ms/step - loss: 0.5640 - accuracy: 0.7573
Epoch 3/3
20741/20741 [==============================] - 52s 2ms/step - loss: 0.5220 - accuracy: 0.7778


In [157]:

print(colored("Testing the Bidirectional GRU  model", "green"))

loss, accuracy = modelbgru.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x3=accuracy
score, accuracy = modelbgru.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y3=accuracy

Testing the Bidirectional GRU  model
20741/20741 [==============================] - 11s 552us/step
Train accuracy: 0.8114362955093384
1749/1749 [==============================] - 1s 538us/step
Test accuracy: 0.7810177206993103


In [159]:
# Building the model
print(colored("Creating the Vanilla RNN model", "yellow"))
modelvrnn = Sequential()
modelvrnn.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
modelvrnn.add(SpatialDropout1D(0.4))
modelvrnn.add(SimpleRNN(256,return_sequences = True))
modelvrnn.add(SimpleRNN(256,return_sequences = False))

modelvrnn.add(Dense(3, activation = 'softmax'))
modelvrnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelvrnn.summary()

Creating the Vanilla RNN model
Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 37, 128)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 37, 256)           98560     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 256)               131328    
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 771       
Total params: 486,659
Trainable params: 486,659
Non-trainable params: 0
_________________________________________________________________


In [160]:
# Training the model
print(colored("Training the Vanilla RNN model", "green"))
history = modelvrnn.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Vanilla RNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 35s 2ms/step - loss: 0.9507 - accuracy: 0.5104
Epoch 2/3
20741/20741 [==============================] - 35s 2ms/step - loss: 0.8106 - accuracy: 0.6154
Epoch 3/3
20741/20741 [==============================] - 35s 2ms/step - loss: 0.5701 - accuracy: 0.7518


In [162]:
print(colored("Testing the Vanilla RNN model", "green"))

loss, accuracy = modelvrnn.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))
x4=accuracy
score, accuracy = modelvrnn.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))
y4=accuracy

Testing the Vanilla RNN model
20741/20741 [==============================] - 11s 532us/step
Train accuracy: 0.8288414478302002
1749/1749 [==============================] - 1s 539us/step
Test accuracy: 0.7878788113594055


In [164]:
from tabulate import tabulate


In [165]:
print(tabulate([
    ['LSTM',x,y], 
    ['GRU',x1,y1],
    ['Bi-LSTM',x2,y2],
    ['Bi-GRU',x3,y3],
    ['V-RNN',x4,y4]], 
    
    headers=['Model', 'Training Accuracy','Testing Accuracy']))

Model      Training Accuracy    Testing Accuracy
-------  -------------------  ------------------
LSTM                0.763464            0.736421
GRU                 0.53522             0.539737
Bi-LSTM             0.805506            0.776444
Bi-GRU              0.811436            0.781018
V-RNN               0.828841            0.787879
